# Vector space models and docs ranking

In [280]:
# AUTHOR : ERRAMI Fatimezahra 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     D:\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     D:\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     D:\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     D:\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     D:\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [281]:
# Collection of documents (corpus)
docs = ["How does the Surface Pro himself 4 compare with iPad Pro?", 
        "Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?",
        "Should I have a hair transplant at age 24? How much would it cost?", 
        "How much cost does hair transplant require?",
        "What but is the best way to send money from China to the US?",
        "Which food not emulsifiers?",
        "What foods fibre?",
        "How are the two wheeler insurance from Bharti Axa insurance?",
        "How their can I start reading?",
        "By scrapping the 500 and 1000 rupee notes, how is RBI planning to fight against issue black money?",
        "Just how do you learn fruity loops?",
        "Why does Batman get kill in Batman v Superman?",
        "When can I buy a SpaceX stock?",
        "Is it gouging and price fixing?",
        "How do Fruity Wrappers work?"
        "Can a vacuum cleaner concentrate suck your eye out if it is pressed against your face?",
        "What you send money to China?",
       ]
docs

['How does the Surface Pro himself 4 compare with iPad Pro?',
 'Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?',
 'Should I have a hair transplant at age 24? How much would it cost?',
 'How much cost does hair transplant require?',
 'What but is the best way to send money from China to the US?',
 'Which food not emulsifiers?',
 'What foods fibre?',
 'How are the two wheeler insurance from Bharti Axa insurance?',
 'How their can I start reading?',
 'By scrapping the 500 and 1000 rupee notes, how is RBI planning to fight against issue black money?',
 'Just how do you learn fruity loops?',
 'Why does Batman get kill in Batman v Superman?',
 'When can I buy a SpaceX stock?',
 'Is it gouging and price fixing?',
 'How do Fruity Wrappers work?Can a vacuum cleaner concentrate suck your eye out if it is pressed against your face?',
 'What you send money to China?']

### Text processing 

In [282]:
def text_preprocessing(corpus) :
    print(corpus)
    # Tokenize the given document
    tokenized = word_tokenize(corpus)
    
    # Lower case all words
    tokenized = [word.lower() for word in tokenized]
    
    # Remove stopwords
    stopwords_en = set(stopwords.words("english"))
    stopwords_en = stopwords_en.union(set(punctuation))
    tokenized_without_sw = [word for word in tokenized if not word in stopwords_en]
    
    # Stemming 
    #porter = PorterStemmer()
    #tokenized_without_sw = [porter.stem(word) for word in tokenized_without_sw]
    
    # POS tagging 
    doc_tagged = pos_tag(tokenized_without_sw)
    
    # Lemmatizer
    wnl = WordNetLemmatizer()
    result = [wnl.lemmatize(word, pos=penn2morphy(tag[:2])) for word, tag in doc_tagged]
    
    return result

In [283]:
def penn2morphy(tag) : 
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[tag]
    except:
        return 'n' # if mapping isn't found, fall back to Noun.

In [284]:
def and_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            p2 += 1
        else:
            p1 += 1
    return result

In [285]:
def or_postings(posting1, posting2):
    p1 = 0
    p2 = 0
    result = list()
    while p1 < len(posting1) and p2 < len(posting2):
        if posting1[p1] == posting2[p2]:
            result.append(posting1[p1])
            p1 += 1
            p2 += 1
        elif posting1[p1] > posting2[p2]:
            result.append(posting2[p2])
            p2 += 1
        else:
            result.append(posting1[p1])
            p1 += 1
    while p1 < len(posting1):
        result.append(posting1[p1])
        p1 += 1
    while p2 < len(posting2):
        result.append(posting2[p2])
        p2 += 1
    return result

In [286]:
docs_preprocessed = []
for doc in docs : 
    docs_preprocessed.append(text_preprocessing(doc))
docs_preprocessed

How does the Surface Pro himself 4 compare with iPad Pro?
Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?
Should I have a hair transplant at age 24? How much would it cost?
How much cost does hair transplant require?
What but is the best way to send money from China to the US?
Which food not emulsifiers?
What foods fibre?
How are the two wheeler insurance from Bharti Axa insurance?
How their can I start reading?
By scrapping the 500 and 1000 rupee notes, how is RBI planning to fight against issue black money?
Just how do you learn fruity loops?
Why does Batman get kill in Batman v Superman?
When can I buy a SpaceX stock?
Is it gouging and price fixing?
How do Fruity Wrappers work?Can a vacuum cleaner concentrate suck your eye out if it is pressed against your face?
What you send money to China?


[['surface', 'pro', '4', 'compare', 'ipad', 'pro'],
 ['microsoft',
  'choose',
  'core',
  'm3',
  'core',
  'i3',
  'home',
  'surface',
  'pro',
  '4'],
 ['hair', 'transplant', 'age', '24', 'much', 'would', 'cost'],
 ['much', 'cost', 'hair', 'transplant', 'require'],
 ['best', 'way', 'send', 'money', 'china', 'u'],
 ['food', 'emulsifier'],
 ['food', 'fibre'],
 ['two', 'wheeler', 'insurance', 'bharti', 'axa', 'insurance'],
 ['start', 'reading'],
 ['scrap',
  '500',
  '1000',
  'rupee',
  'note',
  'rbi',
  'plan',
  'fight',
  'issue',
  'black',
  'money'],
 ['learn', 'fruity', 'loop'],
 ['batman', 'get', 'kill', 'batman', 'v', 'superman'],
 ['buy', 'spacex', 'stock'],
 ['gouge', 'price', 'fixing'],
 ['fruity',
  'wrapper',
  'work',
  'vacuum',
  'clean',
  'concentrate',
  'suck',
  'eye',
  'press',
  'face'],
 ['send', 'money', 'china']]

In [296]:
#Construct docs BOW
docs_bow = [Counter(doc) for doc in docs_preprocessed]
# Gather the set of all unique terms
unique_terms = {term for doc in docs_preprocessed for term in doc}
len(unique_terms)

68

In [297]:
docs_bow

[Counter({'surface': 1, 'pro': 2, '4': 1, 'compare': 1, 'ipad': 1}),
 Counter({'microsoft': 1,
          'choose': 1,
          'core': 2,
          'm3': 1,
          'i3': 1,
          'home': 1,
          'surface': 1,
          'pro': 1,
          '4': 1}),
 Counter({'hair': 1,
          'transplant': 1,
          'age': 1,
          '24': 1,
          'much': 1,
          'would': 1,
          'cost': 1}),
 Counter({'much': 1, 'cost': 1, 'hair': 1, 'transplant': 1, 'require': 1}),
 Counter({'best': 1, 'way': 1, 'send': 1, 'money': 1, 'china': 1, 'u': 1}),
 Counter({'food': 1, 'emulsifier': 1}),
 Counter({'food': 1, 'fibre': 1}),
 Counter({'two': 1, 'wheeler': 1, 'insurance': 2, 'bharti': 1, 'axa': 1}),
 Counter({'start': 1, 'reading': 1}),
 Counter({'scrap': 1,
          '500': 1,
          '1000': 1,
          'rupee': 1,
          'note': 1,
          'rbi': 1,
          'plan': 1,
          'fight': 1,
          'issue': 1,
          'black': 1,
          'money': 1}),
 Counter

In [218]:
# Construct an inverted index
# here as a Python dictionary for ease of interpretability
import collections 

inverted_index = {}

for i, doc in enumerate(docs_preprocessed):
    for term in doc:
        if not term in inverted_index :
            inverted_index[term] = {'df': 1, 'postings_list': set()}
            inverted_index[term]['postings_list'].add(i)
        else : 
            inverted_index[term]['postings_list'].add(i)
            inverted_index[term]['df'] += 1

inverted_index

{'surface': {'df': 2, 'postings_list': {0, 1}},
 'pro': {'df': 3, 'postings_list': {0, 1}},
 '4': {'df': 2, 'postings_list': {0, 1}},
 'compare': {'df': 1, 'postings_list': {0}},
 'ipad': {'df': 1, 'postings_list': {0}},
 'microsoft': {'df': 1, 'postings_list': {1}},
 'choose': {'df': 1, 'postings_list': {1}},
 'core': {'df': 2, 'postings_list': {1}},
 'm3': {'df': 1, 'postings_list': {1}},
 'i3': {'df': 1, 'postings_list': {1}},
 'home': {'df': 1, 'postings_list': {1}},
 'hair': {'df': 2, 'postings_list': {2, 3}},
 'transplant': {'df': 2, 'postings_list': {2, 3}},
 'age': {'df': 1, 'postings_list': {2}},
 '24': {'df': 1, 'postings_list': {2}},
 'much': {'df': 2, 'postings_list': {2, 3}},
 'would': {'df': 1, 'postings_list': {2}},
 'cost': {'df': 2, 'postings_list': {2, 3}},
 'require': {'df': 1, 'postings_list': {3}},
 'best': {'df': 1, 'postings_list': {4}},
 'way': {'df': 1, 'postings_list': {4}},
 'send': {'df': 2, 'postings_list': {4, 15}},
 'money': {'df': 3, 'postings_list': {4,

In [402]:
query = "I want to buy a surface pro laptop"
normalized_query = text_preprocessing(query)

normalized_query

I want to buy a surface pro laptop


['want', 'buy', 'surface', 'pro', 'laptop']

In [403]:
normalized_query = [token for token in normalized_query if token in vocabulary]
normalized_query

['buy', 'surface', 'pro']

In [404]:
def search(q):
    t1 = list(inverted_index[normalized_query[0]]['postings_list'])
    t2 = list(inverted_index[normalized_query[1]]['postings_list'])
    retrieved_docs = or_postings(t1, t2)

    for i in range(1, len(normalized_query)-1) :
        retrieved_docs = or_postings(retrieved_docs, list(inverted_index[normalized_query[i+1]]['postings_list']))
    
    return retrieved_docs

for rs in search(normalized_query):
    print(docs[rs]+"\n")


How does the Surface Pro himself 4 compare with iPad Pro?

Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?

When can I buy a SpaceX stock?



## Create query vector 

In [405]:
vocabulary = unique_terms

In [406]:
def bit_vector_similarity(d_vector, q_vector):
    import numpy as np
    from numpy import linalg
    d = np.array(d_vector)
    q = np.array(q_vector)
    return np.sum(np.multiply(d, q)) / linalg.norm(d)*linalg.norm(q) 

In [407]:
def BM25(Q, D):
    score = 0
    for term in Q :
        score += BM25tf(D[term]) * idf(inverted_index[term]['df'], len(docs))
    return score
    

In [408]:
def BM25tf(c):
    
    # Document length normalization is not needed bcz our documents are small
    #1-b+b*|D|/avgld
    
    # BM25 parameters
    k: float = 1.2
    b: float = 0
    
    return (k+1)*c/(c+k)

In [409]:
def jaccard_similarity(Q, D):
    intersection = set(Q).intersection(set(D))
    union = set(Q).union(D)
    return len(intersection) / len(union)

In [410]:
import math
def idf(df, N):
    return math.log((N+1)/df)

In [411]:
query_vector = []

for term in vocabulary :
    query_vector.append(int(term in normalized_query))
len(query_vector)  

68

In [412]:
docs_vectors = []
doc_ids = search(normalized_query)
for i in doc_ids :
    vector = []
    for term in vocabulary :
        vector.append(int(term in docs_preprocessed[i]))
    docs_vectors.append(vector)


In [413]:
for i, doc_vector in zip(doc_ids, docs_vectors):
    print(f"Doc id : {i} similarity : {bit_vector_similarity(doc_vector, query_vector)}" )
    print(docs[i])

Doc id : 0 similarity : 1.5491933384829666
How does the Surface Pro himself 4 compare with iPad Pro?
Doc id : 1 similarity : 1.1547005383792515
Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?
Doc id : 12 similarity : 1.0
When can I buy a SpaceX stock?


In [414]:
for i in doc_ids:
    print(docs[i])
    print(BM25(normalized_query, Counter(docs_preprocessed[i])))


How does the Surface Pro himself 4 compare with iPad Pro?
4.525142614654917
Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?
3.874667218884377
When can I buy a SpaceX stock?
2.833213344056216


In [415]:
for i in doc_ids :
    print(docs[i])
    print(jaccard_similarity(normalized_query, docs_preprocessed[i]))

How does the Surface Pro himself 4 compare with iPad Pro?
0.3333333333333333
Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?
0.2
When can I buy a SpaceX stock?
0.2
